In [1]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 3.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_6d6a0d32a9b54ab085bd88cc56fbfb7b.csv"
df=pd.read_csv(url)
df.head()

,Case #,Class,I0,PA500,HFS,DA,Area,A/DA,Max IP,DR,P
0,1,car,524.794072,0.187448,0.032114,228.800228,6843.598481,29.910803,60.204880,220.737212,556.828334
1,2,car,330.000000,0.226893,0.265290,121.154201,3163.239472,26.109202,69.717361,99.084964,400.225776
2,3,car,551.879287,0.232478,0.063530,264.804935,11888.391830,44.894903,77.793297,253.785300,656.769449
3,4,car,380.000000,0.240855,0.286234,137.640111,5402.171180,39.248524,88.758446,105.198568,493.701813
4,5,car,362.831266,0.200713,0.244346,124.912559,3290.462446,26.342127,69.389389,103.866552,424.796503


In [3]:
df=df.drop(["Case #"],axis=1)
df.head()

,Class,I0,PA500,HFS,DA,Area,A/DA,Max IP,DR,P
0,car,524.794072,0.187448,0.032114,228.800228,6843.598481,29.910803,60.204880,220.737212,556.828334
1,car,330.000000,0.226893,0.265290,121.154201,3163.239472,26.109202,69.717361,99.084964,400.225776
2,car,551.879287,0.232478,0.063530,264.804935,11888.391830,44.894903,77.793297,253.785300,656.769449
3,car,380.000000,0.240855,0.286234,137.640111,5402.171180,39.248524,88.758446,105.198568,493.701813
4,car,362.831266,0.200713,0.244346,124.912559,3290.462446,26.342127,69.389389,103.866552,424.796503


In [4]:
df.shape

(106, 10)

In [5]:
df.groupby("Class")["Class"].count()

Class
adi    22
car    21
con    14
fad    15
gla    16
mas    18
Name: Class, dtype: int64

In [6]:
X=df.drop(["Class"],axis=1)
X.head()

,I0,PA500,HFS,DA,Area,A/DA,Max IP,DR,P
0,524.794072,0.187448,0.032114,228.800228,6843.598481,29.910803,60.204880,220.737212,556.828334
1,330.000000,0.226893,0.265290,121.154201,3163.239472,26.109202,69.717361,99.084964,400.225776
2,551.879287,0.232478,0.063530,264.804935,11888.391830,44.894903,77.793297,253.785300,656.769449
3,380.000000,0.240855,0.286234,137.640111,5402.171180,39.248524,88.758446,105.198568,493.701813
4,362.831266,0.200713,0.244346,124.912559,3290.462446,26.342127,69.389389,103.866552,424.796503


In [7]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X.head()

,I0,PA500,HFS,DA,Area,A/DA,Max IP,DR,P
0,-0.345766,0.985996,-0.818664,0.201326,-0.026581,0.276930,-0.187452,0.299396,-0.334219
1,-0.605358,1.563752,1.493038,-0.365533,-0.225601,0.113379,-0.069958,-0.374756,-0.540435
2,-0.309671,1.645558,-0.507207,0.390925,0.246221,0.921566,0.029793,0.482536,-0.202616
3,-0.538725,1.768267,1.700676,-0.278719,-0.104528,0.678651,0.165230,-0.340877,-0.417345
4,-0.561605,1.180285,1.285400,-0.345742,-0.218721,0.123400,-0.074009,-0.348258,-0.508080


In [8]:
y=df[["Class"]]
y.head()

,Class
0,car
1,car
2,car
3,car
4,car


In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.35,random_state=42)
print(X_train.shape)
print(X_test.shape)

(68, 9)
(38, 9)


In [10]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np
from sklearn.gaussian_process import GaussianProcessClassifier
BBagging=BalancedBaggingClassifier(GaussianNB(),n_estimators=10,sampling_strategy="auto",n_jobs=-1,
                                   random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BBagging

BalancedBaggingClassifier(base_estimator=GaussianNB(), n_jobs=-1,
                          random_state=0)

In [11]:
from sklearn.metrics import accuracy_score
ytrain_pred=BBagging.predict(X_train)
ytest_pred=BBagging.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,ytrain_pred))
print("Accuracy on test data: ",accuracy_score(y_test,ytest_pred))

Accuracy on train data:  0.6911764705882353
Accuracy on test data:  0.8421052631578947


In [12]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,ytest_pred)

array([[5, 0, 0, 0, 0, 0],
       [0, 9, 0, 0, 0, 0],
       [0, 0, 6, 0, 0, 0],
       [0, 0, 0, 4, 0, 2],
       [0, 0, 0, 1, 3, 1],
       [0, 0, 0, 0, 2, 5]])

In [13]:
y_test.groupby("Class")["Class"].count()

Class
adi    5
car    9
con    6
fad    6
gla    5
mas    7
Name: Class, dtype: int64

In [14]:
df1=df[df["Class"]=="adi"]
df2=df[df["Class"]=="car"]
df3=df[df["Class"]=="con"]
df4=df[df["Class"]=="fad"]
df_sub1=pd.concat([df1,df2,df3,df4],axis=0)
df_sub1.shape

(72, 10)

In [15]:
df5=df[df["Class"]=="gla"]
df6=df[df["Class"]=="mas"]
df_sub2=pd.concat([df5,df6],axis=0)
df_sub2.shape

(34, 10)

In [472]:
X1=df_sub1.drop(["Class"],axis=1)
X1.head()

,I0,PA500,HFS,DA,Area,A/DA,Max IP,DR,P
84,2100.000000,0.061959,-0.045379,390.482507,16640.723620,42.615798,125.900413,380.644703,2073.028685
85,1800.000000,0.034208,0.042586,301.060351,4406.154331,14.635452,67.625328,293.366920,1742.375702
86,2100.000000,0.121649,0.377689,450.551667,35671.606290,79.173176,436.099640,113.198570,2461.450497
87,1666.148786,0.012392,0.058451,72.931050,1402.231665,19.226813,51.854767,58.595763,1746.577504
88,1700.000000,0.043982,0.105127,120.653010,12331.103090,102.203029,120.297318,-9.257696,2212.178196


In [473]:
y1=df_sub1[["Class"]]
y1.head()

,Class
84,adi
85,adi
86,adi
87,adi
88,adi


In [475]:
X1=pd.DataFrame(scaler.fit_transform(X1),columns=X1.columns)
X1.head()

,I0,PA500,HFS,DA,Area,A/DA,Max IP,DR,P
0,1.342988,-0.752540,-1.579109,0.684308,0.285266,0.507156,0.317949,0.815527,1.256821
1,0.966781,-1.111729,-0.737706,0.238347,-0.277307,-0.588946,-0.328862,0.362813,0.847968
2,1.342988,0.020056,2.467639,0.983881,1.160348,1.939254,3.760933,-0.571729,1.737107
3,0.798929,-1.394110,-0.585949,-0.899367,-0.415434,-0.409084,-0.503904,-0.854957,0.853163
4,0.841379,-0.985222,-0.139488,-0.661370,0.087100,2.841425,0.255758,-1.206916,1.428880


In [477]:
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import BalancedBaggingClassifier
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [505]:
BBagging=BalancedBaggingClassifier(base_estimator=RandomForestClassifier(),n_estimators=30,sampling_strategy="not majority",n_jobs=-1,
                                   random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
scores=cross_val_score(BBagging,X1,np.asarray(y1).reshape(y1.shape[0],),cv=10)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9321428571428572 +/- 0.08813544770895049


In [523]:
BForest1=BalancedRandomForestClassifier(n_estimators=250,criterion="gini",max_depth=5,min_samples_split=2,
                                       min_samples_leaf=1,max_features="auto",bootstrap=False,
                                       sampling_strategy="auto",replacement=False,n_jobs=-1,warm_start=False,
                                       random_state=0)
scores=cross_val_score(BForest1,X1,np.asarray(y1).reshape(y1.shape[0],),cv=14)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9333333333333333 +/- 0.1098339294730552


In [528]:
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(BForest1, X1,np.asarray(y1).reshape(y1.shape[0],), cv=14)
print("Confusion matrix: \n",confusion_matrix(y1,y_pred))

Confusion matrix: 
 [[20  0  2  0]
 [ 0 20  0  1]
 [ 1  0 13  0]
 [ 0  1  0 14]]


In [529]:
y1.groupby("Class")["Class"].count()

Class
adi    22
car    21
con    14
fad    15
Name: Class, dtype: int64

In [623]:
print("Sensitivity for adi: ",20/22)
print("Sensitivity for car: ",20/21)
print("Sensitivity for con: ",13/14)
print("Sensitivity for fad: ",14/15)

Sensitivity for adi:  0.9090909090909091
Sensitivity for car:  0.9523809523809523
Sensitivity for con:  0.9285714285714286
Sensitivity for fad:  0.9333333333333333


In [530]:
X2=df_sub2.drop(["Class"],axis=1)
X2.head()

,I0,PA500,HFS,DA,Area,A/DA,Max IP,DR,P
54,470.51642,0.127409,0.066323,150.224022,2657.910367,17.692978,47.560631,142.496468,491.472488
55,223.00000,0.124093,0.079587,33.096335,197.012541,5.952700,30.454874,12.956390,252.483348
56,152.00000,0.165806,0.227591,34.219550,94.354328,2.757322,31.279278,13.877478,180.609553
57,303.00000,0.063355,0.039794,22.567893,102.500309,4.541864,21.830841,5.720501,321.646837
58,250.00000,0.087092,0.092852,29.637510,180.761244,6.099070,26.142097,13.963263,280.122906


In [531]:
X2=pd.DataFrame(scaler.fit_transform(X2),columns=X2.columns)
X2.head()

,I0,PA500,HFS,DA,Area,A/DA,Max IP,DR,P
0,1.780930,0.165728,-0.411575,2.534108,1.823947,1.026979,1.982848,2.411819,1.734226
1,-0.372788,0.089474,-0.266919,-0.677101,-0.549500,-0.615742,0.149357,-0.846750,-0.320722
2,-0.990581,1.048667,1.347136,-0.646307,-0.648510,-1.062845,0.237721,-0.823580,-0.938729
3,0.323317,-1.307174,-0.700887,-0.965752,-0.640654,-0.813149,-0.775016,-1.028768,0.273980
4,-0.137852,-0.761357,-0.122263,-0.771929,-0.565174,-0.595262,-0.312911,-0.821422,-0.083063


In [532]:
y2=df_sub2[["Class"]]
y2.head()

,Class
54,gla
55,gla
56,gla
57,gla
58,gla


In [594]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y2=pd.DataFrame(le.fit_transform(y2),columns=y2.columns)
y2.head()

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


,Class
0,0
1,0
2,0
3,0
4,0


In [595]:
BForest2=BalancedRandomForestClassifier(n_estimators=220,criterion="entropy",max_depth=4,min_samples_split=2,
                                       min_samples_leaf=1,max_features="sqrt",bootstrap=False,
                                       sampling_strategy="all",replacement=False,n_jobs=-1,warm_start=False,
                                       random_state=0)
scores=cross_val_score(BForest2,X2,np.asarray(y2).reshape(y2.shape[0],),cv=14)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8452380952380951 +/- 0.2849692667520434


In [596]:
Forest=RandomForestClassifier(n_estimators=220,criterion="entropy",max_depth=4,min_samples_split=2,
                                       min_samples_leaf=1,max_features="sqrt",bootstrap=False,ccp_alpha=0.02,
                              n_jobs=-1,warm_start=False,random_state=0)
scores=cross_val_score(BForest2,X2,np.asarray(y2).reshape(y2.shape[0],),cv=14)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8452380952380951 +/- 0.2849692667520434


In [618]:
y_pred = cross_val_predict(BForest2, X2,np.asarray(y2).reshape(y2.shape[0],), cv=14)
print("Confusion matrix: \n",confusion_matrix(y2,y_pred))

Confusion matrix: 
 [[13  3]
 [ 3 15]]


In [616]:
y_pred2 = cross_val_predict(Forest, X2,np.asarray(y2).reshape(y2.shape[0],), cv=14)
print("Confusion matrix: \n",confusion_matrix(y2,y_pred2))

Confusion matrix: 
 [[12  4]
 [ 4 14]]


In [619]:
from sklearn.metrics import roc_auc_score
print("AUC after adjusting for the threshold: ",roc_auc_score(y2,y_pred))

AUC after adjusting for the threshold:  0.8229166666666667


In [621]:
y2_label=df_sub2[["Class"]]
y2_label.groupby("Class")["Class"].count()

Class
gla    16
mas    18
Name: Class, dtype: int64

In [622]:
print("From the BalancedRandomForestClassifier model: \n")
print("Sensitivity for gla: ",13/16)
print("Sensitivity for mas: ",15/18)

From the BalancedRandomForestClassifier model: 

Sensitivity for gla:  0.8125
Sensitivity for mas:  0.8333333333333334


In [599]:
y_pred=pd.DataFrame(cross_val_predict(BForest2, X2, np.asarray(y2).reshape(y2.shape[0],), 
                                      cv=14,method='predict_proba')).rename(columns={0:"pred_0",1:"pred_1"})
y_pred.head()

,pred_0,pred_1
0,0.015909,0.984091
1,0.795635,0.204365
2,0.746136,0.253864
3,0.881620,0.118380
4,0.856076,0.143924


In [600]:
y_probs1=y_pred[["pred_1"]]

In [601]:
import numpy as np
thresholds=np.arange(0,1,0.001)

In [602]:
def to_labels(pos_probs,threshold):
    return (pos_probs >= threshold).astype('int')

In [603]:
from sklearn.metrics import f1_score
scores=[f1_score(y_true, to_labels(y_probs1,t),average="micro") for t in thresholds]

In [604]:
ix=np.argmax(scores)
print((thresholds[ix],scores[ix]))

(0.441, 0.8235294117647058)


In [605]:
len(y_probs1[y_probs1["pred_1"]>=0.441])

18

In [606]:
y_pred["pred_0"]=np.where((y_pred.pred_0 >=0.441),1,y_pred.pred_0)
y_pred["pred_0"]=np.where((y_pred.pred_0 < 0.441),0,y_pred.pred_0)
y_pred["pred_1"]=np.where((y_pred.pred_1 >=0.441),1,y_pred.pred_1)
y_pred["pred_1"]=np.where((y_pred.pred_1 <0.441),0,y_pred.pred_1)

In [607]:
len11=len(y_pred[y_pred["pred_1"]==1])
print("The number of predicted positive cases: ",len11)

The number of predicted positive cases:  18


In [608]:
len(y2)-18

16

In [609]:
y_pred["pred_0"]=np.where((y_pred.pred_0 == y_pred.pred_1),0,y_pred.pred_0)

In [610]:
len00=len(y_pred[y_pred["pred_0"]==1])
print("The number of predicted negative cases: ",len00)

The number of predicted negative cases:  16


In [611]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(handle_unknown="ignore")
y_true2=pd.DataFrame(enc.fit_transform(y2).toarray())

In [612]:
y_pred=enc.inverse_transform(y_pred)

In [613]:
from sklearn.metrics import roc_auc_score
print("AUC after adjusting for the threshold: ",roc_auc_score(y2,y_pred))

AUC after adjusting for the threshold:  0.8229166666666667


In [614]:
print("Confusion_matrix after adjusting the threshold: \n",confusion_matrix(y2,y_pred))

Confusion_matrix after adjusting the threshold: 
 [[13  3]
 [ 3 15]]


No more room for further improvement through adjusting the threshold.